<a href="https://colab.research.google.com/github/Swathi2201/ShapeAI_Bootcamp_BWD/blob/main/Money_bill_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def generate_monthly_bill(item_list, target_month):
    #Target Year and Month
    year_input, mon_input = target_month.split('-')
    year = int(year_input)
    month = int(mon_input)

    #Days in the target month
    if month in (1,3,5,7,8,10,12):
        days_in_month = 31
    elif month in (4,6,9,11):
        days_in_month = 30
    else:
        if (year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)):
            days_in_month = 29
        else:
            days_in_month = 28

    month_start = (year, month, 1)
    month_end   = (year, month, days_in_month)

    segments = []
    for item in item_list:
        #(YYYY,MM,DD) format
        y1, m1, d1 = item['start_date'].split('-')
        y2, m2, d2 = item['stop_date'].split('-')
        start = (int(y1), int(m1), int(d1))
        stop  = (int(y2), int(m2), int(d2))

        # Finding Overlap with the target month
        active_start = start if start > month_start else month_start
        active_end   = stop  if stop  < month_end   else month_end
        if active_start > active_end:
            continue

        qty  = int(item['qty'])
        rate = float(item['rate'])

        # Days active in the month
        days_active = active_end[2] - active_start[2] + 1

        # Total Amount
        amount = rate * qty * (days_active / days_in_month)

        # Billing period
        billing_period = (
            f"{active_start[0]:04d}-{active_start[1]:02d}-{active_start[2]:02d}"
            f" to "
            f"{active_end[0]:04d}-{active_end[1]:02d}-{active_end[2]:02d}"
        )

        segments.append({
            'item_code':      item['item_code'],
            'rate':           rate,
            'qty':            qty,
            'amount':         amount,
            'billing_period': billing_period
        })

    grouped = {}
    for item in segments:
        key = (item['item_code'], item['rate'], item['billing_period'])
        if key not in grouped:
            grouped[key] = {'qty': 0, 'amount': 0.0}
        grouped[key]['qty']+= item['qty']
        grouped[key]['amount']+= item['amount']
    line_items = []
    total_revenue = 0.0
    for (item_code, rate, period), vals in grouped.items():
        amt = round(vals['amount'], 2)
        line_items.append({
            'item_code':      item_code,
            'rate':           rate,
            'qty':            vals['qty'],
            'amount':         amt,
            'billing_period': period
        })
        total_revenue += amt
    line_items.sort(key=lambda x: (x['item_code'], x['rate'], x['billing_period']))

    return {
        'line_items':    line_items,
        'total_revenue': round(total_revenue, 2)
    }
